# <center> *Phase2(Part2):* **Embedded Search**

In [1]:
import sys
sys.path.append('D:\AUT Courses\Information Retrieval\Project')

In [2]:
from Phase1.PositionalBooleanSearch import read_documents, preprocess
from Phase2.VectorizedSearch import load_dictionary, tfidf, retrieve_docs
from Phase2.VectorizedSearch import Dictionary, Term
from gensim.models import Word2Vec
from numpy.linalg import norm
import numpy as np
import multiprocessing
import pickle
import time

In [3]:
def prepare_training_data():
    documents, titles = read_documents()
    # print(len(documents))  # 7562
    documents_tokens = preprocess(documents)
    with open('..\\Phase2\\training_data.pkl', 'wb') as output:
        pickle.dump(documents_tokens, output)

In [4]:
def load_training_data():
    with open('..\\Phase2\\training_data.pkl', 'rb') as input:
        return pickle.load(input)

In [5]:
def build_model():
    training_data = load_training_data()
    cores = multiprocessing.cpu_count()

    w2v_model = Word2Vec(min_count=1, window=5, vector_size=300, alpha=0.03, workers=cores - 1)
    w2v_model.build_vocab(training_data)

    w2v_model.train(training_data, total_examples=w2v_model.corpus_count, epochs=20)
    w2v_model.save("w2v_model_300.model")

In [6]:
# prepare_training_data()
# build_model()

In [7]:
def load_model(model):
    return Word2Vec.load(model)

In [8]:
def build_docs_tfidf_dict(dictionary):
    documents_tokens = load_training_data()
    docs_tfidf = []

    for doc_id in range(len(documents_tokens)):
        tokens = documents_tokens[doc_id]
        doc_tfidf = {}
        for token in tokens:
            term = dictionary.get_term(token)
            doc_tfidf[token] = term.tfidf_weight(doc_id, dictionary.N)
        docs_tfidf.append(doc_tfidf)

    return docs_tfidf

In [9]:
def documents_embedding(model, dictionary):
    docs_embedding = []
    docs_tfidf = build_docs_tfidf_dict(dictionary)

    for document in docs_tfidf:
        doc_vector = np.zeros(300)
        weights_sum = 0

        for term, weight in document.items():
            weights_sum += weight
            try:
                doc_vector += model.wv[term] * weight
            except KeyError:
                continue

        if weights_sum == 0:
            docs_embedding.append(doc_vector)
            continue

        docs_embedding.append(doc_vector / weights_sum)

    return docs_embedding

In [10]:
def build_documents_vectors(model, dictionary, file_name):
    with open("..\\Phase2\\" + file_name, 'wb') as output:
        pickle.dump(documents_embedding(model, dictionary), output)

In [11]:
def load_documents_vectors(file_name):
    with open("..\\Phase2\\" + file_name, 'rb') as input:
        return pickle.load(input)

In [12]:
def query_embedding(query, model, dictionary):
    query_terms = preprocess([query])[0]
    query_vector = np.zeros(300)
    weights_sum = 0

    for qt in query_terms:
        try:
            qt_tf = query_terms.count(qt)
            qt_df = dictionary.get_term(qt).df
            weight = tfidf(qt_tf, qt_df, dictionary.N)
            query_vector += model.wv[qt] * weight
        except KeyError:
            # print(qt)
            continue
        weights_sum += weight

    if weights_sum == 0:
        return query_vector

    return query_vector / weights_sum

In [13]:
def embedded_search(query_vector, docs_vectors, dictionary, k):
    similarity = {}
    # print(norm(query_vector))
    for doc_id in range(len(docs_vectors)):
        doc_vector = docs_vectors[doc_id]
        vectors_norm = norm(query_vector) * norm(doc_vector)
        if vectors_norm == 0:
            similarity[doc_id] = 0
            continue
        similarity_score = np.dot(query_vector, doc_vector) / vectors_norm
        similarity[doc_id] = (similarity_score + 1) / 2

    return retrieve_docs(similarity, dictionary, k)

In [14]:
model_name = "w2v_model_300.model"
d2v_file_name = "d2v_model_300.pkl"

# model_name = "w2v_150k_hazm_300_v2.model"
# d2v_file_name = "d2v_150k_hazm_300_v2.pkl"

In [15]:
model = load_model(model_name)
dictionary = load_dictionary()

In [16]:
# build_documents_vectors(model, dictionary, d2v_file_name)
docs_vectors = load_documents_vectors(d2v_file_name)

In [17]:
while True:
    print(">> Please Enter your Query: ", end='')
    query = input()

    if not query:
        break

    query_vector = query_embedding(query, model, dictionary)

    start_time = time.time()
    print(">> Top 10 Results for «%s» : " % query)
    print("====================")
    for result in embedded_search(query_vector, docs_vectors, dictionary, k=10):
        print(result.strip())
    print("===================================================")
    print(">> Retrieval Time: --- %s seconds ---" % (time.time() - start_time))
    print("===================================================")

>> Please Enter your Query: تفریغ بودجه
>> Top 10 Results for «تفریغ بودجه» : 
آغاز جلسه غیرعلنی مجلس برای بررسی گزارش تفریغ بودجه ۹۹
سی‌ودومین گزارش تفریغ بودجه ۲۰ مهر توسط بذرپاش در مجلس قرائت می‌شود
بذرپاش اعلام کرد: رکوردشکنی در تفریغ بودجه 99/ نظارت دیوان محاسبات بر اجرای بودجه در مقاطع ۲ ماهه
نظارت دیوان محاسبات بر اجرای بودجه سالانه در مقاطع ۲ ماهه را باید به فال نیک گرفت
قالیباف: تا سه ماه آینده گزارش نهایی تفریغ بودجه 99 در صحن قرائت خواهد شد
گام بلند دیوان محاسبات برای آنلاین شدن «تفریغ بودجه»/ پایانی بر گزارش‌های تاریخ گذشته از دخل و خرج‌ها
دولت در شش ماهه نخست امسال 115 هزار میلیارد تومان برای هزینه‌های جاری اوراق فروخت
دولت دوازدهم یارانه نقدی سه دهک بالای درآمدی را قطع نکرد
تهیه دومین گزارش نظارت مستمر ۲ ماهه بر اجرای بودجه سال ۱۴۰۰
گزارش تفریغ بودجه ۹۹ به تفکیک دستگاهی و استانی به زودی به مجلس می‌رسد
>> Retrieval Time: --- 0.07848858833312988 seconds ---
>> Please Enter your Query: وزارت نفت
>> Top 10 Results for «وزارت نفت» : 
کاهش 30 درصدی سطح مخازن نیروگاه‌ها نسبت به 